# Восстановление золота из руды

Необходимо подготовить прототип модели машинного обучения для компании «Цифры», которая разрабатывает решения для эффективной работы промышленных предприятий.
Модель должна предсказать коэффициент восстановления золота из золотосодержащей руды. У нас есть данные с параметрами добычи и очистки.
Модель поможет оптимизировать производство, чтобы не запускать предприятие с убыточными характеристиками.

В процессе выполнения проекта необходимо:

- Подготовить данные;
- Провести исследовательский анализ данных;
- Построить и обучить модель.

***Технологический процесс***

Когда добытая руда проходит первичную обработку, получается дроблёная смесь. Её отправляют на флотацию (обогащение) и двухэтапную очистку.


**Стадии:**

1. Флотация

Во флотационную установку подаётся смесь золотосодержащей руды. После обогащения получается черновой концентрат и «отвальные хвосты», то есть остатки продукта с низкой концентрацией ценных металлов.
На стабильность этого процесса влияет непостоянное и неоптимальное физико-химическое состояние флотационной пульпы (смеси твёрдых частиц и жидкости).

2. Очистка

Черновой концентрат проходит две очистки. На выходе получается финальный концентрат и новые отвальные хвосты.


***Описание данных***

**Технологический процесс**

* `Rougher feed` — исходное сырье
* `Rougher additions` (или `reagent additions`) — флотационные реагенты: Xanthate, Sulphate, Depressant
    * `Xanthat **` — ксантогенат (промотер, или активатор флотации);
    * `Sulphate` — сульфат (на данном производстве сульфид натрия);
    * `Depressant` — депрессант (силикат натрия).
* `Rougher process` (англ. «грубый процесс») — флотация
* `Rougher tails` — отвальные хвосты
* `Float banks` — флотационная установка
* `Cleaner process` — очистка
* `Rougher Au` — черновой концентрат золота
* `Final Au` — финальный концентрат золота

**Параметры этапов**

* `air amount` — объём воздуха
* `fluid levels` — уровень жидкости
* `feed size` — размер гранул сырья
* `feed rate` — скорость подачи


Нужно спрогнозировать сразу две величины:

1. эффективность обогащения чернового концентрата `rougher.output.recovery`;
2. эффективность обогащения финального концентрата `final.output.recovery`.


Данные находятся в трёх файлах:

* `gold_recovery_train_new_8ds.csv` — обучающая выборка;
* `gold_recovery_test_new_8ds.csv` — тестовая выборка;
* `gold_recovery_full_new_8ds.csv` — исходные данные.

Данные индексируются датой и временем получения информации (признак `date`). Соседние по времени параметры часто похожи.

Некоторые параметры недоступны, потому что замеряются и/или рассчитываются значительно позже. Из-за этого в тестовой выборке отсутствуют некоторые признаки, которые могут быть в обучающей. Также в тестовом наборе нет целевых признаков.

Исходный датасет содержит обучающую и тестовую выборки со всеми признаками.

Данные сырые. Необхоима их проверка и предобработка.

## Загрузка и подготовка данных

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important; }</style>"))

***Выгрузка библиотек***

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

***Константы***

In [3]:
RANDOM_SEED = 0

In [4]:
# Based on: https://stackoverflow.com/a/17303428 (для красивого оформления текста)
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

***Функции***

In [5]:
#Рандоманая выгрузка строк
show_sample = lambda x: display(x.sample(5, random_state=RANDOM_SEED).reset_index(drop=True))

In [6]:
def com_info(data):
    '''
    Функция для изучения общей информации: input: data
                                           output: - вывод рандомных строк,
                                                   - типов данных,
                                                   - % пропусков
                                                   - статистики
                                                   - наличие дубликатов
                                                   - размерность данных
                                                   
    '''
    name = [x for x in globals() if globals()[x] is data][0]
    print(color.UNDERLINE + 'Общая информация по данным '+ name + color.END)
    show_sample(data)
    print(data.info(memory_usage=True))
    display(pd.DataFrame(round((data.isna().mean()), 2)).style.background_gradient('coolwarm')\
                                                                .set_caption('% пропусков').format({0:'{:.2%}'.format}))
    display(data.describe().T.style.set_caption('Статистика'))
    print('Количество дублей: ', data.duplicated().sum())
    print('Количество строк: ', data.shape[0], ', количество столбцов: ', data.shape[1])
    print('******************************************************************')
    print('')

In [7]:
def downcast_func(data):
    name = [x for x in globals() if globals()[x] is data][0]
    print(color.UNDERLINE + 'Информация по типам данных '+ name + color.END)
    float_features = data.select_dtypes(include='float').columns
    data[float_features] = data[float_features].apply(pd.to_numeric, downcast='float')
    print(data.info(memory_usage=True))

***Загрука данных***

In [8]:
df_train = pd.read_csv('../datasets/gold_recovery_train_new_8ds.csv')
df_test = pd.read_csv('../datasets/gold_recovery_test_new_8ds.csv')
df_full = pd.read_csv('../datasets/gold_recovery_full_new_8ds.csv')

## Изучение общей информации

In [9]:
df_list = [df_train, df_test, df_full]
for i in df_list:
    com_info(i)

Общая информация по данным df_train


,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.output.concentrate_ag,primary_cleaner.output.concentrate_pb,primary_cleaner.output.concentrate_sol,primary_cleaner.output.concentrate_au,primary_cleaner.output.tail_ag,primary_cleaner.output.tail_pb,primary_cleaner.output.tail_sol,primary_cleaner.output.tail_au,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,primary_cleaner.state.floatbank8_d_air,primary_cleaner.state.floatbank8_d_level,rougher.calculation.sulfate_to_au_concentrate,rougher.calculation.floatbank10_sulfate_to_au_feed,rougher.calculation.floatbank11_sulfate_to_au_feed,rougher.calculation.au_pb_ratio,rougher.input.feed_ag,rougher.input.feed_pb,rougher.input.feed_rate,rougher.input.feed_size,rougher.input.feed_sol,rougher.input.feed_au,rougher.input.floatbank10_sulfate,rougher.input.floatbank10_xanthate,rougher.input.floatbank11_sulfate,rougher.input.floatbank11_xanthate,rougher.output.concentrate_ag,rougher.output.concentrate_pb,rougher.output.concentrate_sol,rougher.output.concentrate_au,rougher.output.recovery,rougher.output.tail_ag,rougher.output.tail_pb,rougher.output.tail_sol,rougher.output.tail_au,rougher.state.floatbank10_a_air,rougher.state.floatbank10_a_level,rougher.state.floatbank10_b_air,rougher.state.floatbank10_b_level,rougher.state.floatbank10_c_air,rougher.state.floatbank10_c_level,rougher.state.floatbank10_d_air,rougher.state.floatbank10_d_level,rougher.state.floatbank10_e_air,rougher.state.floatbank10_e_level,rougher.state.floatbank10_f_air,rougher.state.floatbank10_f_level,secondary_cleaner.output.tail_ag,secondary_cleaner.output.tail_pb,secondary_cleaner.output.tail_sol,secondary_cleaner.output.tail_au,secondary_cleaner.state.floatbank2_a_air,secondary_cleaner.state.floatbank2_a_level,secondary_cleaner.state.floatbank2_b_air,secondary_cleaner.state.floatbank2_b_level,secondary_cleaner.state.floatbank3_a_air,secondary_cleaner.state.floatbank3_a_level,secondary_cleaner.state.floatbank3_b_air,secondary_cleaner.state.floatbank3_b_level,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2018-06-08 12:59:59,4.068389,11.448127,5.983031,45.728369,63.185926,7.436988,2.577877,10.211517,2.042844,99.732267,7.252426,6.825,0.702993,5.747566,13.588954,7.378889,34.649302,17.597198,4.702111,6.517120,2.368953,1603.268907,-500.115801,1471.571680,-499.576251,1602.685484,-499.972861,1027.751721,-500.811690,42913.665507,2785.559395,2785.149436,1.318954,5.969035,3.907571,499.626977,70.977362,40.129088,5.153906,7.387563,5.597894,7.386242,5.612319,11.436856,11.132484,28.501758,16.771833,90.669868,3.317203,0.288877,18.156484,0.666596,1197.020398,-299.987078,1202.540003,-500.426514,1202.957760,-499.831297,1099.868778,-500.408752,1000.158975,-499.684249,1000.345311,-500.801754,15.455878,11.476563,NaN,5.244986,36.009503,-500.230128,30.014005,-499.948950,37.007576,-500.146875,27.975799,-499.881781,30.026765,-485.043326,22.022924,-466.815927,25.026081,-495.662399,21.919914,-500.075170,24.989364,-501.420340
1,2016-06-24 08:59:59,4.672297,10.013952,8.417600,45.142223,70.512351,8.217110,1.566327,11.871072,2.433252,107.117324,5.006355,7.690,1.024347,8.001182,9.58721

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14149 entries, 0 to 14148
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                14149 non-null  object 
 1   final.output.concentrate_ag                         14148 non-null  float64
 2   final.output.concentrate_pb                         14148 non-null  float64
 3   final.output.concentrate_sol                        13938 non-null  float64
 4   final.output.concentrate_au                         14149 non-null  float64
 5   final.output.recovery                               14149 non-null  float64
 6   final.output.tail_ag                                14149 non-null  float64
 7   final.output.tail_pb                                14049 non-null  float64
 8   final.output.tail_sol                               14144 non-null  float64


,0
date,0.00%
final.output.concentrate_ag,0.00%
final.output.concentrate_pb,0.00%
final.output.concentrate_sol,1.00%
final.output.concentrate_au,0.00%
final.output.recovery,0.00%
final.output.tail_ag,0.00%
final.output.tail_pb,1.00%
final.output.tail_sol,0.00%
final.output.tail_au,0.00%


,count,mean,std,min,25%,50%,75%,max
final.output.concentrate_ag,14148.000000,5.142034,1.369586,0.000000,4.211620,4.994652,5.859540,16.001945
final.output.concentrate_pb,14148.000000,10.132960,1.654930,0.000000,9.297355,10.297144,11.170603,17.031899
final.output.concentrate_sol,13938.000000,9.202849,2.790516,0.000000,7.484645,8.845462,10.487508,18.124851
final.output.concentrate_au,14149.000000,44.003792,4.905261,0.000000,43.276111,44.872436,46.166425,52.756638
final.output.recovery,14149.000000,66.518832,10.295402,0.000000,62.545817,67.432775,72.346428,100.000000
final.output.tail_ag,14149.000000,9.607035,2.319069,0.000000,7.997429,9.480270,11.003707,19.552149
final.output.tail_pb,14049.000000,2.597298,0.971843,0.000000,1.905973,2.592022,3.241723,5.639565
final.output.tail_sol,14144.000000,10.512122,3.003617,0.000000,8.811324,10.514621,11.933009,22.317730
final.output.tail_au,14149.000000,2.918421,0.903712,0.000000,2.368607,2.851025,3.434764,8.197408
primary_cleaner.input.sulfate,14129.000000,133.320659,39.431659,0.003112,107.006651,133.018328,159.825396,250.127834


...

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.output.concentrate_ag,primary_cleaner.output.concentrate_pb,primary_cleaner.output.concentrate_sol,primary_cleaner.output.concentrate_au,primary_cleaner.output.tail_ag,primary_cleaner.output.tail_pb,primary_cleaner.output.tail_sol,primary_cleaner.output.tail_au,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,primary_cleaner.state.floatbank8_d_air,primary_cleaner.state.floatbank8_d_level,rougher.calculation.sulfate_to_au_concentrate,rougher.calculation.floatbank10_sulfate_to_au_feed,rougher.calculation.floatbank11_sulfate_to_au_feed,rougher.calculation.au_pb_ratio,rougher.input.feed_ag,rougher.input.feed_pb,rougher.input.feed_rate,rougher.input.feed_size,rougher.input.feed_sol,rougher.input.feed_au,rougher.input.floatbank10_sulfate,rougher.input.floatbank10_xanthate,rougher.input.floatbank11_sulfate,rougher.input.floatbank11_xanthate,rougher.output.concentrate_ag,rougher.output.concentrate_pb,rougher.output.concentrate_sol,rougher.output.concentrate_au,rougher.output.recovery,rougher.output.tail_ag,rougher.output.tail_pb,rougher.output.tail_sol,rougher.output.tail_au,rougher.state.floatbank10_a_air,rougher.state.floatbank10_a_level,rougher.state.floatbank10_b_air,rougher.state.floatbank10_b_level,rougher.state.floatbank10_c_air,rougher.state.floatbank10_c_level,rougher.state.floatbank10_d_air,rougher.state.floatbank10_d_level,rougher.state.floatbank10_e_air,rougher.state.floatbank10_e_level,rougher.state.floatbank10_f_air,rougher.state.floatbank10_f_level,secondary_cleaner.output.tail_ag,secondary_cleaner.output.tail_pb,secondary_cleaner.output.tail_sol,secondary_cleaner.output.tail_au,secondary_cleaner.state.floatbank2_a_air,secondary_cleaner.state.floatbank2_a_level,secondary_cleaner.state.floatbank2_b_air,secondary_cleaner.state.floatbank2_b_level,secondary_cleaner.state.floatbank3_a_air,secondary_cleaner.state.floatbank3_a_level,secondary_cleaner.state.floatbank3_b_air,secondary_cleaner.state.floatbank3_b_level,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2017-02-08 14:59:59,5.295988,8.195301,13.379583,47.758663,65.280176,9.925318,3.387088,7.605304,3.382011,88.025736,8.082156,6.780000,1.349388,8.509287,7.432810,6.704215,33.850846,17.925953,3.713285,5.571651,3.788193,1794.481420,-350.265470,1799.470760,-350.491718,1798.338009,-351.657929,1538.980253,-349.106166,37855.335517,3107.930038,3107.466213,2.486664,8.419499,3.456962,391.128017,NaN,29.196869,8.596304,9.501066,5.699917,9.499393,5.700741,13.238094,7.467266,35.876975,24.887139,82.764867,6.059697,0.573929,11.876489,2.074698,1499.646078,-299.953656,1399.068872,-500.023359,1298.497113,-500.326487,1199.215979,-499.876276,999.566016,-500.533131,999.870553,-500.549211,15.811448,4.603052,6.923786,3.267344,33.999922,-339.998859,30.030993,-350.122742,36.972836,-500.594480,29.987487,-500.319561,24.937716,-499.827034,22.955855,-499.981726,23.018173,-499.465128,20.014626,-500.003639,24.991929,-599.773615
1,2016-10-29 13:59:59,3.020476,9.071122,16.816693,49.797229,63.724514,8.165338,3.167601,10.705567,3.591239,149.481074,9.316742,7.260000,0.804833,5.416631,10.64017

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19439 entries, 0 to 19438
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                19439 non-null  object 
 1   final.output.concentrate_ag                         19438 non-null  float64
 2   final.output.concentrate_pb                         19438 non-null  float64
 3   final.output.concentrate_sol                        19228 non-null  float64
 4   final.output.concentrate_au                         19439 non-null  float64
 5   final.output.recovery                               19439 non-null  float64
 6   final.output.tail_ag                                19438 non-null  float64
 7   final.output.tail_pb                                19338 non-null  float64
 8   final.output.tail_sol                               19433 non-null  float64


,0
date,0.00%
final.output.concentrate_ag,0.00%
final.output.concentrate_pb,0.00%
final.output.concentrate_sol,1.00%
final.output.concentrate_au,0.00%
final.output.recovery,0.00%
final.output.tail_ag,0.00%
final.output.tail_pb,1.00%
final.output.tail_sol,0.00%
final.output.tail_au,0.00%


,count,mean,std,min,25%,50%,75%,max
final.output.concentrate_ag,19438.000000,5.168470,1.372348,0.000000,4.251240,5.066094,5.895527,16.001945
final.output.concentrate_pb,19438.000000,9.978895,1.669240,0.000000,9.137262,10.102433,11.035769,17.031899
final.output.concentrate_sol,19228.000000,9.501224,2.787537,0.000000,7.722820,9.218961,10.947813,19.615720
final.output.concentrate_au,19439.000000,44.076513,5.129784,0.000000,43.402215,45.011244,46.275313,52.756638
final.output.recovery,19439.000000,67.050208,10.125840,0.000000,63.299712,68.172738,72.686642,100.000000
final.output.tail_ag,19438.000000,9.688589,2.328642,0.000000,8.062878,9.743623,11.134294,19.552149
final.output.tail_pb,19338.000000,2.705795,0.949077,0.000000,2.040119,2.748730,3.333216,5.804178
final.output.tail_sol,19433.000000,10.583728,2.868782,0.000000,8.938947,10.622456,12.104271,22.317730
final.output.tail_au,19439.000000,3.042467,0.922808,0.000000,2.461138,2.984909,3.571351,8.245022
primary_cleaner.input.sulfate,19415.000000,144.624774,44.464071,0.003112,114.107029,143.232103,175.075656,265.983123


Количество дублей:  0
Количество строк:  19439 , количество столбцов:  87
******************************************************************



#### Выводы

***df_train***

- Типы данных не оптимизированы, необходимо понизить разрядность признаков
- В данных есть пропуски, больше всего в признаке `secondary_cleaner.output.tail_sol` (11%). Пропусков в целевых признаках нет
- Некоторые признаки отрицательные - например, `primary_cleaner.state.floatbank8_b_level`, `primary_cleaner.state.floatbank8_c_level`, `rougher.state.floatbank10_a_level` и т.д.
- Полных дублей в данных нет
- Количество строк:  14149 , количество столбцов:  87



***df_test***

- Типы данных не оптимизированы, необходимо понизить разрядность признаков
- В данных небольшое количество пропусков по некоторым признакам - например `rougher.input.feed_sol ` , `rougher.input.floatbank11_sulfate` (меньше 1%)
- Некоторые признаки отрицательные - например, `primary_cleaner.state.floatbank8_a_level`, `rougher.state.floatbank10_b_level` и т.д.
- Полных дублей в данных нет.
- Количество строк:  5290 , количество столбцов:  53

***df_full***

- Типы данных не оптимизированы, необходимо понизить разрядность признаков
- В данных есть пропуски, больше всего в признаке  `secondary_cleaner.output.tail_sol `  (9%). Пропусков в целевых признаках нет
- Некоторые признаки отрицательные - например, `primary_cleaner.state.floatbank8_a_level`, `rougher.state.floatbank10_b_level` и т.д.
- Полных дублей в данных нет
- Количество строк:  19439 , количество столбцов:  87

## Предобработка

***Оптимизация типов данных***

In [10]:
for i in df_list:
     downcast_func(i)

Информация по типам данных df_train
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14149 entries, 0 to 14148
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                14149 non-null  object 
 1   final.output.concentrate_ag                         14148 non-null  float32
 2   final.output.concentrate_pb                         14148 non-null  float32
 3   final.output.concentrate_sol                        13938 non-null  float32
 4   final.output.concentrate_au                         14149 non-null  float32
 5   final.output.recovery                               14149 non-null  float32
 6   final.output.tail_ag                                14149 non-null  float32
 7   final.output.tail_pb                                14049 non-null  float32
 8   final.output.tail_sol                   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19439 entries, 0 to 19438
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                19439 non-null  object 
 1   final.output.concentrate_ag                         19438 non-null  float32
 2   final.output.concentrate_pb                         19438 non-null  float32
 3   final.output.concentrate_sol                        19228 non-null  float32
 4   final.output.concentrate_au                         19439 non-null  float32
 5   final.output.recovery                               19439 non-null  float32
 6   final.output.tail_ag                                19438 non-null  float32
 7   final.output.tail_pb                                19338 non-null  float32
 8   final.output.tail_sol                               19433 non-null  float32
